In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [2]:
# Load the train data
train_data = pd.read_csv('train.csv')


In [3]:
# Separate features and target
X = train_data.iloc[:, 1:].values
y = train_data.iloc[:, 0].values



In [4]:
# Normalize the pixel values
X = X / 255.0


In [6]:
X = X.reshape(-1, 28, 28, 1)


In [7]:
# One-hot encode the labels
y = to_categorical(y)

In [8]:
# Split the data into training, validation, and testing sets (70%, 15%, 15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=9)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=9)


In [11]:
# Define the model
input_shape = (28, 28, 1)
inputs = Input(shape=input_shape)

In [12]:
# Base model - MobileNetV2 expects at least 32x32 input, so we will resize using UpSampling
x = tf.keras.layers.UpSampling2D(size=(8, 8))(inputs)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [13]:
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(y.shape[1], activation='softmax')(x)

In [14]:
model = Model(inputs, predictions)

In [15]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [16]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=32, epochs=10)

Epoch 1/10


ValueError: Exception encountered when calling Functional.call().

[1mInput 0 of layer "Conv1" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (None, 224, 224, 1)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(None, 224, 224, 1), dtype=float32)
  • training=True
  • mask=None